## Setup

In [1]:
import os
import json
import glob
import torch
import re
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
#import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
from utils.data_processing import (
    load_edge_scores_into_dictionary,
    compute_ged,
    compute_weighted_ged,
    compute_gtd,
    compute_jaccard_similarity_to_reference,
    compute_jaccard_similarity,
    compute_weighted_jaccard_similarity,
    compute_weighted_jaccard_similarity_to_reference
)
from utils.visualization import plot_graph_metric

In [2]:
# =============================================================================
#import kaleido
#pio.renderers.default = 'png' # USE IF MAKING GRAPHS FOR NOTEBOOK EXPORT
# =============================================================================

In [3]:
TASK = 'ioi'
PERFORMANCE_METRIC = 'logit_diff'
MODEL_NAME = 'pythia-160m'

### Plotting Code

In [4]:


def load_graphs_for_models(target_directory, TASK):
    df_list = []
    for root, dirs, files in os.walk(target_directory):
        for dir in dirs:
            folder_path = f'results/graphs/{dir}/{TASK}'
            if os.path.isdir(folder_path):
                df = load_edge_scores_into_dictionary(folder_path)
                df['subfolder'] = dir  # Add the subfolder name as a new column
                df_list.append(df)
                clear_output()

    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

## Retrieve & Process Data

### Circuit Data

In [5]:
# only needs to be run if another model is added or graph files change in some way
# all_graphs = load_graphs_for_models('results/graphs', TASK)
# all_graphs = all_graphs[all_graphs['in_circuit'] == True]
# all_graphs.to_pickle(f'results/all_minimal_graphs_{TASK}.pkl')

In [6]:
# load dataframe from pickle file
all_graphs = pd.read_pickle(f'results/all_minimal_graphs_{TASK}.pkl')
all_graphs = all_graphs[all_graphs['checkpoint'] >= 4000]
all_graphs.rename(columns={'subfolder': 'model'}, inplace=True)
all_graphs.sort_values(by=['model', 'checkpoint'], inplace=True)

all_graphs.head()

,edge,score,in_circuit,checkpoint,model
48588208,m1->m6,0.000349,True,4000,pythia-160m
48588319,a4.h11->a6.h6<v>,0.000355,True,4000,pythia-160m
48588428,a4.h8->a6.h6<v>,0.000450,True,4000,pythia-160m
48588655,m4->m6,0.000431,True,4000,pythia-160m
48589427,m1->a8.h9<v>,0.010071,True,4000,pythia-160m


In [7]:
# Group by checkpoint and subfolder and sum the number of edges
subgraph_df = all_graphs.groupby(['checkpoint', 'model']).sum().reset_index()

subgraph_df.head()


,checkpoint,model,edge,score,in_circuit
0,4000,pythia-160m,m1->m6a4.h11->a6.h6<v>a4.h8->a6.h6<v>m4->m6m1-...,0.345562,78
1,4000,pythia-160m-alldropout,m0->a7.h4<k>m0->a7.h4<v>m0->a7.h10<v>m0->m8m0-...,0.142632,47
2,4000,pythia-160m-data-seed1,m0->a8.h1<v>m0->a8.h9<v>m2->a8.h9<v>input->m0m...,0.292908,7
3,4000,pythia-160m-data-seed2,m0->a7.h10<v>m1->a8.h9<v>m0->a8.h10<v>m0->a8.h...,0.256348,22
4,4000,pythia-160m-data-seed3,m0->a8.h9<v>m0->a8.h9<k>m0->a8.h1<v>m2->a8.h9<...,0.241760,8


In [8]:
# sort this
models = subgraph_df['model'].unique().tolist()
models.sort()

### Performance Data

In [9]:
perf_metrics_by_model = dict()

for model in models:
    # check if file exists
    if os.path.isfile(f'results/backup/{model}/nmh_backup_metrics.pt'):
        perf_metrics = torch.load(f'results/backup/{model}/nmh_backup_metrics.pt')
        perf_metrics_by_model[model] = {checkpoint: perf_metrics[checkpoint][PERFORMANCE_METRIC] for checkpoint in perf_metrics.keys()}
    else:
        print(f"File for {model} does not exist")
        perf_metrics_by_model[model] = {checkpoint: 0 for checkpoint in subgraph_df[subgraph_df['model'] == model]['checkpoint'].unique()}

File for pythia-160m-weight-seed3 does not exist


In [10]:
# df_list = []
# for root, dirs, files in os.walk(target_directory):
#     for dir in dirs:
#         folder_path = f'results/backup/{dir}/{TASK}'
#         if os.path.isdir(folder_path):
#             df = load_edge_scores_into_dictionary(folder_path)
#             df['subfolder'] = dir  # Add the subfolder name as a new column
#             df_list.append(df)
#             clear_output()

# combined_df = pd.concat(df_list, ignore_index=True)

## Visualize Results

### Graph Size

In [12]:
for model in models:
    model_df = subgraph_df[subgraph_df['model'] == model].copy()
    plot_graph_metric(
        model_df, 
        'in_circuit', 
        perf_metrics_by_model[model], 
        f'Graph Size for {model}',
        left_y_title="Edge Count", 
        y_range=1000, 
        x_axis_col='checkpoint', 
        log_x=True, 
        disable_title=False # Optional: Set to True to disable the title for publishing
    )

### Graph Similarity

In [15]:
def compute_weighted_jaccard_similarity(df):
    # Ensure the dataframe is sorted by checkpoint
    df = df.sort_values(by='checkpoint')

    # Normalize the scores by dividing by the sum of absolute scores within each checkpoint
    df['normalized_abs_score'] = df.groupby('checkpoint')['score'].transform(lambda x: x.abs() / x.abs().sum())

    # Get the unique checkpoints
    checkpoints = df['checkpoint'].unique()

    # Initialize a list to store the results
    results = []

    # Iterate over pairs of adjacent checkpoints
    for i in range(len(checkpoints) - 1):
        # Get the data for each checkpoint
        df_1 = df[(df['checkpoint'] == checkpoints[i]) & (df['in_circuit'] == True)]
        df_2 = df[(df['checkpoint'] == checkpoints[i + 1]) & (df['in_circuit'] == True)]

        # Create dictionaries mapping edges to their normalized absolute scores
        scores_1 = dict(zip(df_1['edge'], df_1['normalized_abs_score']))
        scores_2 = dict(zip(df_2['edge'], df_2['normalized_abs_score']))

        # Calculate the weighted intersection and union
        weighted_intersection = sum(min(scores_1.get(edge, 0), scores_2.get(edge, 0)) for edge in set(scores_1) | set(scores_2))
        weighted_union = sum(max(scores_1.get(edge, 0), scores_2.get(edge, 0)) for edge in set(scores_1) | set(scores_2))

        # Calculate the weighted Jaccard similarity
        weighted_jaccard_similarity = weighted_intersection / weighted_union if weighted_union != 0 else 0

        # Append the results for this pair of checkpoints
        results.append({
            'checkpoint_1': checkpoints[i],
            'checkpoint_2': checkpoints[i + 1],
            'jaccard_similarity': weighted_jaccard_similarity
        })

    # Convert the results to a DataFrame and return
    return pd.DataFrame(results)


In [16]:
for model in models:
    model_df = all_graphs[all_graphs['model'] == model].copy()
    weighted_jaccard_results = compute_weighted_jaccard_similarity(model_df)
    plot_graph_metric(
        weighted_jaccard_results, 
        'jaccard_similarity', 
        perf_metrics_by_model[model], 
        f'Weighted Jaccard Similarity to Previous Checkpoint for {model}', 
        left_y_title="Weighted Jaccard Similarity",
        y_range=1, 
        x_axis_col='checkpoint_2', 
        log_x=False,
        disable_title=True
    )

In [12]:
comparison_checkpoint = 5000

for model in models:
    model_df = all_graphs[all_graphs['model'] == model].copy()
    jaccard_reference_results = compute_weighted_jaccard_similarity_to_reference(model_df, comparison_checkpoint)
    plot_graph_metric(
        jaccard_reference_results, 
        'jaccard_similarity', 
        perf_metrics_by_model[model], 
        f'Weighted Jaccard Similarity to Step {comparison_checkpoint} for {model}', 
        left_y_title="Weighted Jaccard Similarity",
        y_range=1, 
        x_axis_col='checkpoint', 
        log_x=True,
        disable_title=True
    )

In [13]:
comparison_checkpoint = 143000

for model in models:
    model_df = all_graphs[all_graphs['model'] == model].copy()
    jaccard_reference_results = compute_weighted_jaccard_similarity_to_reference(model_df, comparison_checkpoint)
    plot_graph_metric(
        jaccard_reference_results, 
        'jaccard_similarity', 
        perf_metrics_by_model[model], 
        f'Weighted Jaccard Similarity to Step {comparison_checkpoint} for {model}', 
        left_y_title="Weighted Jaccard Similarity",
        y_range=1, 
        x_axis_col='checkpoint', 
        log_x=True,
        disable_title=True
    )

In [14]:
from utils.data_processing import compute_ewma_weighted_jaccard_similarity
for model in models:
    model_df = all_graphs[all_graphs['model'] == model].copy()
    jaccard_reference_results = compute_ewma_weighted_jaccard_similarity(model_df, alpha=0.1)
    plot_graph_metric(
        jaccard_reference_results, 
        'ewma_change_rate',
        perf_metrics_by_model[model],
        f'EWMA Weighted Change Rate for {model}',
        left_y_title="Weighted Change Rate",
        y_range=1.0,
        x_axis_col='checkpoint_2',
        log_x=False,
        disable_title=True
    )

In [90]:
plot_graph_metric??

Signature:
plot_graph_metric(
    df,
    metric,
    perf_metric_dict,
    title,
    y_range,
    x_axis_col='checkpoint',
    log_x=True,
)
Docstring: <no docstring>
Source:   
def plot_graph_metric(df, metric, perf_metric_dict, title, y_range, x_axis_col='checkpoint', log_x=True):
    # Add a new column for the performance metric by mapping the checkpoint_2 values using the perf_metric_dict
    df['perf_metric'] = df[x_axis_col].map(perf_metric_dict)

    # Interpolate missing values
    df['perf_metric'] = df['perf_metric'].interpolate(method='linear')

    # plot weighted additions, deletions, and total weighted GED over time
    fig = px.line(df, width=1200, x=x_axis_col, y=[metric], title=title, log_x=log_x)

    # Specify colors for each line
    colors = {metric: 'lightblue', 'perf_metric': 'black'}

    # Update each trace with the specified color
    for i, trace in enumerate(fig.data):
        fig.data[i].update(line=dict(color=colors[trace.name]))


    # Convert to a go.